In [ ]:
import numpy as np
import pandas as pd
from os.path import exists
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import itertools as it
import seaborn as sns
import plotly.graph_objects as go
import plotly.io as pio
import pingouin as pg
import scipy.stats as stats
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.proportion import proportion_confint
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multitest import multipletests
from statsmodels.stats.anova import AnovaRM
from statsmodels.multivariate.manova import MANOVA
%matplotlib inline

In [ ]:
# Create a dataframe of all subjects in the final sample
phenotype_df = pd.read_csv('[Insert filepath to phenotype data]', sep='\t', low_memory=False)
phenotype_df = phenotype_df[phenotype_df['INT_NUM'] == 1]
SUBJID_df = pd.concat([pd.read_csv('../results/youth_SUBJIDs'), pd.read_csv('../results/early_adult_SUBJIDs')], axis=0)
phenotype_df = pd.merge(phenotype_df, SUBJID_df['SUBJID'], how='inner', on='SUBJID')

In [ ]:
# Create four groupings for the race variable
phenotype_df['Race'] = phenotype_df['Race'].fillna('OT')
def race_func(x):
    if x in {'EA', 'AA'}:
        return x
    if x in {'OT', 'PI', 'AI', 'Eskimo/Alaskan'}:
        return 'OT'
    return 'Mixed'
phenotype_df['Race'] = phenotype_df['Race'].apply(race_func)

In [ ]:
# define lists for the different categories of variables
ID_vars = ['SUBJID', 'INT_NUM', 'INT_TYPE', 'Sex']
ADD = ['ADD011', 'ADD012', 'ADD013', 'ADD014', 'ADD015', 'ADD016', 'ADD020', 'ADD021', 'ADD022']
AGR = ['AGR001', 'AGR002', 'AGR003', 'AGR004', 'AGR005', 'AGR006', 'AGR007', 'AGR008']
CDD = ['CDD001', 'CDD002', 'CDD003', 'CDD004', 'CDD005', 'CDD006', 'CDD007', 'CDD008']
DEP = ['DEP001', 'DEP002', 'DEP004', 'DEP006']
GAD = ['GAD001', 'GAD002']
MAN = ['MAN001', 'MAN002', 'MAN003', 'MAN004', 'MAN005', 'MAN006', 'MAN007']
OCD = ['OCD001', 'OCD002', 'OCD003', 'OCD004', 'OCD005', 'OCD006', 'OCD007', 'OCD008', 'OCD011', 'OCD012', 'OCD013', 'OCD014', 'OCD015', 'OCD016', 'OCD017', 'OCD018', 'OCD019']
ODD = ['ODD001', 'ODD002', 'ODD003', 'ODD005', 'ODD006']
PAN = ['PAN001', 'PAN003', 'PAN004']
PHB = ['PHB001', 'PHB002', 'PHB003', 'PHB004', 'PHB005', 'PHB006', 'PHB007', 'PHB008']
PSY = ['PSY001', 'PSY029', 'PSY050', 'PSY060', 'PSY070', 'PSY071']
SIP = ['SIP003', 'SIP004', 'SIP005', 'SIP006', 'SIP007', 'SIP008', 'SIP009', 'SIP010', 'SIP011', 'SIP012', 'SIP013', 'SIP014']
PTD = ['PTD001', 'PTD002', 'PTD003', 'PTD004', 'PTD006', 'PTD007', 'PTD008', 'PTD009']
SEP = ['SEP500', 'SEP508', 'SEP509', 'SEP510', 'SEP511']
SOC = ['SOC001', 'SOC002', 'SOC003', 'SOC004', 'SOC005']
SUI = ['SUI001', 'SUI002']

# dictionary for grouping the disorder variables
disorders = {
    'ADD': ADD,
    'AGR': AGR,
    'CDD': CDD,
    'DEP': DEP,
    'GAD': GAD,
    'MAN': MAN,
    'OCD': OCD,
    'ODD': ODD,
    'PAN': PAN,
    'PHB': PHB,
    'PSY': PSY,
    'SIP': SIP,
    'PTD': PTD,
    'SEP': SEP,
    'SOC': SOC,
    'SUI': SUI
}

for disorder in disorders.keys():
    if disorder == 'SIP':
        phenotype_df[disorder] = phenotype_df[disorders[disorder]].mean(axis=1)
    else:
        phenotype_df[disorder] = phenotype_df[disorders[disorder]].sum(axis=1)

In [ ]:
# Split the dataframe into the youth and early adult age groups
youth_phenotype_df = phenotype_df[phenotype_df['INT_TYPE'] == 'MP']
early_adult_phenotype_df = phenotype_df[phenotype_df['INT_TYPE'] == 'AP']

# Analysis of demographic information (i.e., gender, race, and age) before community detection

In [ ]:
# Descriptive statistics for gender
print('Entire Sample:')
print(phenotype_df['Sex'].value_counts())

print('\nYouth age group:')
print(youth_phenotype_df['Sex'].value_counts())

print('\nEarly adult age group:')
print(early_adult_phenotype_df['Sex'].value_counts())

In [ ]:
# Descriptive statistics for race
print('Entire Sample:')
print(phenotype_df['Race'].value_counts())

print('\nYouth age group:')
print(youth_phenotype_df['Race'].value_counts())

print('\nEarly adult age group:')
print(early_adult_phenotype_df['Race'].value_counts())

In [ ]:
# Descriptive statistics for age at enrollment
print('Entire Sample:')
print(phenotype_df['age_at_cnb'].mean())
print(phenotype_df['age_at_cnb'].std())
print(phenotype_df['age_at_cnb'].skew())
print(phenotype_df['age_at_cnb'].kurtosis())

print('\nYouth age group:')
print(youth_phenotype_df['age_at_cnb'].mean())
print(youth_phenotype_df['age_at_cnb'].std())
print(youth_phenotype_df['age_at_cnb'].skew())
print(youth_phenotype_df['age_at_cnb'].kurtosis())

print('\nEarly adult age group:')
print(early_adult_phenotype_df['age_at_cnb'].mean())
print(early_adult_phenotype_df['age_at_cnb'].std())
print(early_adult_phenotype_df['age_at_cnb'].skew())
print(early_adult_phenotype_df['age_at_cnb'].kurtosis())

# Analysis of psychopathology symptoms before community detection

In [ ]:
# Create DataFrames with block membership columns for both the youth and early adult age groups

youth_membership_list_filepath = '../results/youth_multiplex_Pearson_memberships.csv'

youth_membership_list = pd.read_csv(youth_membership_list_filepath)['memberships'].values.tolist()
        
# save the number of blocks
Q_youth = max(youth_membership_list)
        
# add the block memberships to the phenotype dataframe
youth_phenotype_df['Block'] = youth_membership_list


early_adult_membership_list_filepath = '../results/early_adult_multiplex_Pearson_memberships.csv'

early_adult_membership_list = pd.read_csv(early_adult_membership_list_filepath)['memberships'].values.tolist()
        
# save the number of blocks
Q_early_adult = max(early_adult_membership_list)
        
# add the block memberships to the phenotype dataframe
early_adult_phenotype_df['Block'] = early_adult_membership_list

In [ ]:
# Descriptive statistics for each psychopathology domain
print('Entire Sample:')
for disorder in disorders.keys():
    print(disorder)
    print(round(phenotype_df[disorder].mean(), 3))
    print(round(phenotype_df[disorder].std(), 3))
    print(round(phenotype_df[disorder].skew(), 3))
    print(round(phenotype_df[disorder].kurtosis(), 3))
    print()
    
print('\nYouth age group:')
for disorder in disorders.keys():
    print(disorder)
    print(round(youth_phenotype_df[disorder].mean(), 3))
    print(round(youth_phenotype_df[disorder].std(), 3))
    print(round(youth_phenotype_df[disorder].skew(), 3))
    print(round(youth_phenotype_df[disorder].kurtosis(), 3))
    print()

print('\nEarly adult age group:')
for disorder in disorders.keys():
    print(disorder)
    print(round(early_adult_phenotype_df[disorder].mean(), 3))
    print(round(early_adult_phenotype_df[disorder].std(), 3))
    print(round(early_adult_phenotype_df[disorder].skew(), 3))
    print(round(early_adult_phenotype_df[disorder].kurtosis(), 3))
    print()

# Radar Plots for both the youth and early adult age groups
domains = list(disorders.keys())
binary_domains = domains.copy()
binary_domains.remove('SIP')

fill_colors = ['rgba(102, 194, 165, 0.25)', 'rgba(252, 141, 98, 0.25)', 'rgba(141, 160, 203, 0.25)', 'rgba(231, 138, 195, 0.25)']
fill_color_cycle = it.cycle(fill_colors)

line_colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3']
line_color_cycle = it.cycle(line_colors)

filepath = '../results/symptom_scores_unclustered'
trace_list = [] # Maintain a list of traces

avg_scores = youth_phenotype_df[binary_domains].mean(axis=0)
for domain in binary_domains:
    avg_scores[domain] = avg_scores[domain] / len(disorders[domain]) 
trace_list.append(go.Scatterpolar(
    name = 'Youth', # Name for radar plot (age group)
    r=avg_scores.values,  # Average values of the variables
    theta=avg_scores.index,  # Variable names as theta values
    mode='lines',  # Set the mode to 'lines' to remove dots on points
    fill='toself',  # Fill area inside the radar plot
    fillcolor=next(fill_color_cycle),  # Fill color is set to next color in cycle
    line=dict(color=next(line_color_cycle))  # Set the line color
))

avg_scores = early_adult_phenotype_df[binary_domains].mean(axis=0)
for domain in binary_domains:
    avg_scores[domain] = avg_scores[domain] / len(disorders[domain]) 
trace_list.append(go.Scatterpolar(
    name = 'Early Adults', # Name for radar plot (age group)
    r=avg_scores.values,  # Average values of the variables
    theta=avg_scores.index,  # Variable names as theta values
    mode='lines',  # Set the mode to 'lines' to remove dots on points
    fill='toself',  # Fill area inside the radar plot
    fillcolor=next(fill_color_cycle),  # Fill color is set to next color in cycle
    line=dict(color=next(line_color_cycle))  # Set the line color
))

fig = go.Figure(data=trace_list)
    
fig.update_layout(
    title={
        'text': f'Psychopathology Domains with Binary Item-level Responses',
        'x': 0.555,  # Set x to 0.555 for alignment slightly right of center
        'y': 0.9,  # Set y to adjust the vertical position
        'font': {'size': 16}
    },
    polar=dict(
        radialaxis=dict(
            visible=True,  # Show the radial axis
            range=[0, 0.6],  # Set the range for the radial axis
            dtick=0.25
        )
    ),
    legend={
        'title': {'text': 'Age Group'},  # Set the title for the legend
        'title_font': {'size': 14},  # Set the font size for the legend title
        'x': 0.865,  # Adjust the horizontal position of the legend (0-1)
    }
)
fig.show()

image = pio.to_image(fig, format='png', width=8, height=8, scale=1)
with open(filepath + '.png', 'wb') as f:
    f.write(image)
    
# Violin Plot for both the youth and early adult age groups (SIP Vars)
filepath = '../results/SIP_scores_violin_unclustered'

youth_phenotype_df['age_group'] = 'Youth'
early_adult_phenotype_df['age_group'] = 'Early Adults'

cols = ['age_group', 'Block'] + domains

data = pd.concat([youth_phenotype_df[cols], early_adult_phenotype_df[cols]])

fig, ax = plt.subplots(figsize=(6,3))

sns.set_palette(sns.color_palette('Set2'))

sns.violinplot(x='Age Group', y='SIP', data=data.rename(columns={'age_group': 'Age Group'}), scale='count', linewidth=2, hue='Age Group')

plt.ylim(0, 6) # Set y-axis range to 0-6

# Set title and axis labels
ax.set(xlabel='Age Group', ylabel='Mean Item-Level Response Value\n(Ranging from 0 to 6)')
ax.set_title('Psychosis Prodrome (SIP) Domain')

plt.tick_params(axis='x', which='both', bottom=False, top=False) # Remove x-axis ticks

sns.despine()

fig.savefig(filepath, bbox_inches='tight', dpi=300)

In [ ]:
# T-tests for comparing the domains across age groups
file = open('../results/adj_ttest_output_before_clustering.txt', 'w')

t_stats = []
p_vals = []

domains = list(disorders.keys())

for domain in domains:
    sample1 = np.array(youth_phenotype_df[domain].tolist())
    sample2 = np.array(early_adult_phenotype_df[domain].tolist())

    # Perform the t-test
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)
    t_stats.append(t_statistic)
    p_vals.append(p_value)

# Correct for multiple comparisons using the Benjamini-Hochberg method
adj_p_vals = multipletests(p_vals, method='fdr_bh')[1]
    
for i,domain in enumerate(domains):
    file.write(f'{domain} Psychopathology Domain\n\n')
    
    t_statistic = t_stats[i]
    p_value = p_vals[i]
    adj_p_value = adj_p_vals[i]
    
    file.write(f't={t_statistic}\n')
    file.write(f'p={p_value}\n')
    file.write(f'BH-adj p={adj_p_value}\n')

    # Set the critical value
    alpha = 0.05

    # Determine if the null hypothesis is rejected or not
    if adj_p_value < alpha:
        file.write('Reject null hypothesis\n')
    else:
        file.write('Fail to reject null hypothesis\n')
    
    file.write('\n\n\n')
    
file.close()

In [ ]:
# One way ANOVAs and two-tailed t-tests for comparing the different psychopathology domains (other than the SIP domain)

print('Youth age group:\n')

youth_data = pd.melt(youth_phenotype_df, id_vars=['Block'], var_name='Domains', value_vars=binary_domains)

model = ols('value ~ C(Domains)', data=youth_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

tukey = pairwise_tukeyhsd(endog=youth_data['value'], groups=youth_data['Domains'], alpha=0.05)
print(tukey)
p_vals = tukey.pvalues

rej, adj_p_vals, _, _ = multipletests(p_vals, method='fdr_bh')
print("p-values:", p_vals)
print("Adjusted p-values:", adj_p_vals)
print("Reject decisions:", rej)

print('Early adult age group:\n')

early_adult_data = pd.melt(early_adult_phenotype_df, id_vars=['Block'], var_name='Domains', value_vars=binary_domains)

model = ols('value ~ C(Domains)', data=youth_data).fit()
anova_table = sm.stats.anova_lm(model, typ=2)
print(anova_table)

tukey = pairwise_tukeyhsd(endog=early_adult_data['value'], groups=early_adult_data['Domains'], alpha=0.05)
print(tukey)
p_vals = tukey.pvalues

rej, adj_p_vals, _, _ = multipletests(p_vals, method='fdr_bh')
print("p-values:", p_vals)
print("Adjusted p-values:", adj_p_vals)
print("Reject decisions:", rej)

# Analysis of block-wise demographic information (i.e., gender, race, and age) following community detection

In [ ]:
# Plots comparing the gender distributions across blocks and age groups
filepath = '../results/gender_comparisons'

gender_df = pd.concat([youth_phenotype_df[['INT_TYPE', 'Block', 'Sex']], early_adult_phenotype_df[['INT_TYPE', 'Block', 'Sex']]], ignore_index=True)

# Function to calculate proportion and confidence interval
def calc_proportion_and_ci(group):
    n_total = group.shape[0]
    n_females = group[group['Sex'] == 'F'].shape[0]
    proportion_females = n_females / n_total
    
    # Confidence Interval using the Wilson method
    ci_low, ci_high = proportion_confint(n_females, n_total, method='wilson')
    
    return proportion_females, ci_low, ci_high

# Calculate proportions and CIs for each category
results = []

# (1) Total proportion of females
total_F, total_ci_low, total_ci_high = calc_proportion_and_ci(gender_df)
results.append(('All Ages', total_F, total_ci_low, total_ci_high))

# (2) Proportion among youth
youth = gender_df[gender_df['INT_TYPE'] == 'MP']
youth_F, youth_ci_low, youth_ci_high = calc_proportion_and_ci(youth)
results.append(('Youth (Y)', youth_F, youth_ci_low, youth_ci_high))

# (3) Proportion among Block 1 in youth
youth_1 = youth[youth['Block'] == 1]
youth_1_F, youth_1_ci_low, youth_1_ci_high = calc_proportion_and_ci(youth_1)
results.append(('Y: Block 1', youth_1_F, youth_1_ci_low, youth_1_ci_high))

# (4) Proportion among Block 2 in youth
youth_2 = youth[youth['Block'] == 2]
youth_2_F, youth_2_ci_low, youth_2_ci_high = calc_proportion_and_ci(youth_2)
results.append(('Y: Block 2', youth_2_F, youth_2_ci_low, youth_2_ci_high))

# (5) Proportion among early adults
early_adults = gender_df[gender_df['INT_TYPE'] == 'AP']
early_adults_F, early_adults_ci_low, early_adults_ci_high = calc_proportion_and_ci(early_adults)
results.append(('Early Adults (EA)', early_adults_F, early_adults_ci_low, early_adults_ci_high))

# (6) Proportion among Block 1 in early adults
early_adults_1 = early_adults[early_adults['Block'] == 1]
early_adults_1_F, early_adults_1_ci_low, early_adults_1_ci_high = calc_proportion_and_ci(early_adults_1)
results.append(('EA: Block 1', early_adults_1_F, early_adults_1_ci_low, early_adults_1_ci_high))

# (7) Proportion among Block 2 in early adults
early_adults_2 = early_adults[early_adults['Block'] == 2]
early_adults_2_F, early_adults_2_ci_low, early_adults_2_ci_high = calc_proportion_and_ci(early_adults_2)
results.append(('EA: Block 2', early_adults_2_F, early_adults_2_ci_low, early_adults_2_ci_high))

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Grouping', 'Proportion of Females', 'CI Lower', 'CI Upper'])

# Get tab10 colors from Matplotlib
colors = plt.cm.tab10.colors

# Manually assign tab10 colors to each bar
custom_tab10_colors = [colors[0], colors[1], colors[1], colors[1], colors[2], colors[2], colors[2]]

# Plot
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(x='Grouping', y='Proportion of Females', data=results_df, errorbar=None, palette=custom_tab10_colors)

# Add error bars for confidence intervals
for i in range(len(results_df)):
    plt.errorbar(x=i,
                 y=results_df['Proportion of Females'].iloc[i],
                 yerr=[[results_df['Proportion of Females'].iloc[i] - results_df['CI Lower'].iloc[i]],
                       [results_df['CI Upper'].iloc[i] - results_df['Proportion of Females'].iloc[i]]],
                 fmt='none', c='black', capsize=5)

# Add embellishments for the bars representing individual blocks
for i, patch in enumerate(ax.patches):
    if i == 1 or i == 4:  # Customize the pattern of the 1st and 5th bars
        patch.set_hatch('x')
    elif i == 2 or i == 5:  # Customize the pattern of the 2nd and 6th bars
        patch.set_hatch('/')
    elif i == 3 or i == 6:  # Customize the pattern of the 3rd and 7th bars
        patch.set_hatch('\\')

plt.axhline(y = 0.5, color = 'black', linestyle = '--')
        
plt.title('Proportion of Females by Age Group and Block')
plt.ylim(0, 1)
plt.ylabel('Proportion of Females')
plt.xlabel('Grouping (Age Group and/or Block)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

fig.savefig(filepath, bbox_inches='tight', dpi=300)

In [ ]:
# Test associations between gender, age group, and block

# Create a contingency table
contingency_table = pd.crosstab(index=[gender_df['Sex'], gender_df['INT_TYPE']], columns=gender_df['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between gender, age group, and block.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between gender, age group, and block.")


# Youth
# Create a contingency table
contingency_table = pd.crosstab(index=[gender_df[gender_df['INT_TYPE'] == 'MP']['Sex']], columns=gender_df[gender_df['INT_TYPE'] == 'MP']['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between gender and block among youth.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between gender and block among youth.")
    
# Early Adults
# Create a contingency table
contingency_table = pd.crosstab(index=[gender_df[gender_df['INT_TYPE'] == 'AP']['Sex']], columns=gender_df[gender_df['INT_TYPE'] == 'AP']['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between gender and block among early adults.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between gender and block among early adults.")

In [ ]:
# Block-wise descriptive statistics for gender
print('Youth age group:\n')
for i in range(1, Q_youth+1):
    print(f'Block {i}')
    block_df = youth_phenotype_df[youth_phenotype_df['Block'] == i]
    print(block_df['Sex'].value_counts())
    print()

    
print('Early adult age group:\n')
for i in range(1, Q_early_adult+1):
    print(f'Block {i}')
    block_df = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == i]
    print(block_df['Sex'].value_counts())
    print()

In [ ]:
# Plots comparing the race distributions across blocks and age groups
filepath = '../results/race_comparisons'

race_df = pd.concat([youth_phenotype_df[['INT_TYPE', 'Block', 'Race']], early_adult_phenotype_df[['INT_TYPE', 'Block', 'Race']]], ignore_index=True)

# Function to calculate proportion and confidence interval
def calc_proportion_and_ci(group):
    n_total = group.shape[0]
    n_blacks = group[group['Race'] == 'AA'].shape[0]
    proportion_blacks = n_blacks / n_total
    
    # Confidence Interval using the Wilson method
    ci_low, ci_high = proportion_confint(n_blacks, n_total, method='wilson')
    
    return proportion_blacks, ci_low, ci_high

# Calculate proportions and CIs for each category
results = []

# (1) Total proportion of black subjects
total_AA, total_ci_low, total_ci_high = calc_proportion_and_ci(race_df)
results.append(('All Ages', total_AA, total_ci_low, total_ci_high))

# (2) Proportion among youth
youth = race_df[race_df['INT_TYPE'] == 'MP']
youth_AA, youth_ci_low, youth_ci_high = calc_proportion_and_ci(youth)
results.append(('Youth (Y)', youth_AA, youth_ci_low, youth_ci_high))

# (3) Proportion among Block 1 in youth
youth_1 = youth[youth['Block'] == 1]
youth_1_AA, youth_1_ci_low, youth_1_ci_high = calc_proportion_and_ci(youth_1)
results.append(('Y: Block 1', youth_1_AA, youth_1_ci_low, youth_1_ci_high))

# (4) Proportion among Block 2 in youth
youth_2 = youth[youth['Block'] == 2]
youth_2_AA, youth_2_ci_low, youth_2_ci_high = calc_proportion_and_ci(youth_2)
results.append(('Y: Block 2', youth_2_AA, youth_2_ci_low, youth_2_ci_high))

# (5) Proportion among early adults
early_adults = race_df[race_df['INT_TYPE'] == 'AP']
early_adults_AA, early_adults_ci_low, early_adults_ci_high = calc_proportion_and_ci(early_adults)
results.append(('Early Adults (EA)', early_adults_AA, early_adults_ci_low, early_adults_ci_high))

# (6) Proportion among Block 1 in early adults
early_adults_1 = early_adults[early_adults['Block'] == 1]
early_adults_1_AA, early_adults_1_ci_low, early_adults_1_ci_high = calc_proportion_and_ci(early_adults_1)
results.append(('EA: Block 1', early_adults_1_AA, early_adults_1_ci_low, early_adults_1_ci_high))

# (7) Proportion among Block 2 in early adults
early_adults_2 = early_adults[early_adults['Block'] == 2]
early_adults_2_AA, early_adults_2_ci_low, early_adults_2_ci_high = calc_proportion_and_ci(early_adults_2)
results.append(('EA: Block 2', early_adults_2_AA, early_adults_2_ci_low, early_adults_2_ci_high))

# Convert results to DataFrame
results_df = pd.DataFrame(results, columns=['Grouping', 'Proportion of Black Participants', 'CI Lower', 'CI Upper'])

# Get tab10 colors from Matplotlib
colors = plt.cm.tab10.colors

# Manually assign tab10 colors to each bar
custom_tab10_colors = [colors[0], colors[1], colors[1], colors[1], colors[2], colors[2], colors[2]]

# Plot
fig, ax = plt.subplots(figsize=(6,4))
sns.barplot(x='Grouping', y='Proportion of Black Participants', data=results_df, errorbar=None, palette=custom_tab10_colors)

# Add error bars for confidence intervals
for i in range(len(results_df)):
    plt.errorbar(x=i,
                 y=results_df['Proportion of Black Participants'].iloc[i],
                 yerr=[[results_df['Proportion of Black Participants'].iloc[i] - results_df['CI Lower'].iloc[i]],
                       [results_df['CI Upper'].iloc[i] - results_df['Proportion of Black Participants'].iloc[i]]],
                 fmt='none', c='black', capsize=5)

# Add embellishments for the bars representing individual blocks
for i, patch in enumerate(ax.patches):
    if i == 1 or i == 4:  # Customize the pattern of the 1st and 5th bars
        patch.set_hatch('x')
    elif i == 2 or i == 5:  # Customize the pattern of the 2nd and 6th bars
        patch.set_hatch('/')
    elif i == 3 or i == 6:  # Customize the pattern of the 3rd and 7th bars
        patch.set_hatch('\\')

plt.axhline(y = 0.5, color = 'black', linestyle = '--')
        
plt.title('Proportion of Black Participants by Age Group and Block')
plt.ylim(0, 1)
plt.ylabel('Proportion of Black Subjects')
plt.xlabel('Grouping (Age Group and/or Block)')
plt.xticks(rotation=30)
plt.tight_layout()
plt.show()

fig.savefig(filepath, bbox_inches='tight', dpi=300)

In [ ]:
# Test associations between race, age group, and block

# Create a contingency table
contingency_table = pd.crosstab(index=[race_df['Race'], race_df['INT_TYPE']], columns=race_df['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between race, age group, and block.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between race, age group, and block.")

# Youth
# Create a contingency table
contingency_table = pd.crosstab(index=[race_df[race_df['INT_TYPE'] == 'MP']['Race']], columns=race_df[race_df['INT_TYPE'] == 'MP']['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between race and block among youth.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between race and block among youth.")
    
# Early Adults
# Create a contingency table
contingency_table = pd.crosstab(index=[race_df[race_df['INT_TYPE'] == 'AP']['Race']], columns=race_df[race_df['INT_TYPE'] == 'AP']['Block'])

print("Contingency Table:")
print(contingency_table)

# Conduct the Chi-Square test
chi2_stat, p_val, dof, expected = chi2_contingency(contingency_table)

# Output the results
print("\nChi-Square Test Results:")
print(f"Chi-Squared Statistic: {chi2_stat}")
print(f"P-value: {p_val}")
print(f"Degrees of Freedom: {dof}")
print("Expected Frequencies Table:")
print(expected)

# Interpret the result
alpha = 0.05
if p_val < alpha:
    print("Reject the null hypothesis: There is a significant association between race and block among early adults.")
else:
    print("Fail to reject the null hypothesis: There is no significant association between race and block among early adults.")

In [ ]:
# Block-wise Descriptive Statistics for Race
print('Youth age group:\n')
for i in range(1, Q_youth+1):
    print(f'Block {i}')
    block_df = youth_phenotype_df[youth_phenotype_df['Block'] == i]
    print(block_df['Race'].value_counts())
    print()

    
print('Early adult age group:\n')
for i in range(1, Q_early_adult+1):
    print(f'Block {i}')
    block_df = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == i]
    print(block_df['Race'].value_counts())
    print()

In [ ]:
# Block-wise descriptive statistics for age at enrollment
print('Youth age group:\n')
for i in range(1, Q_youth+1):
    print(f'Block {i}')
    block_df = youth_phenotype_df[youth_phenotype_df['Block'] == i]
    print(block_df['age_at_cnb'].mean())
    print(block_df['age_at_cnb'].std())
    print(block_df['age_at_cnb'].skew())
    print(block_df['age_at_cnb'].kurtosis())
    print()

    
print('Early adult age group:\n')
for i in range(1, Q_early_adult+1):
    print(f'Block {i}')
    block_df = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == i]
    print(block_df['age_at_cnb'].mean())
    print(block_df['age_at_cnb'].std())
    print(block_df['age_at_cnb'].skew())
    print(block_df['age_at_cnb'].kurtosis())
    print()

# Block-wise analysis of psychopathology symptoms following community detection

In [ ]:
# Block-wise visualizations and descriptive statistics for each psychopathology domain

fill_colors = ['rgba(102, 194, 165, 0.25)', 'rgba(252, 141, 98, 0.25)', 'rgba(141, 160, 203, 0.25)', 'rgba(231, 138, 195, 0.25)']
fill_color_cycle = it.cycle(fill_colors)

line_colors = ['#66c2a5', '#fc8d62', '#8da0cb', '#e78ac3']
line_color_cycle = it.cycle(line_colors)

print('Youth age group:\n')

# Radar plots and descriptive statistics for the youth age group
filepath = '../results/youth_symptom_scores_radar_plot'
trace_list = [] # Maintain a list of traces
for i in range(1, Q_youth+1):
    block_df = youth_phenotype_df[youth_phenotype_df['Block'] == i]
    avg_scores = block_df[binary_domains].mean(axis=0)
    for domain in binary_domains:
        avg_scores[domain] = avg_scores[domain] / len(disorders[domain])
    
    trace_list.append(go.Scatterpolar(
        name = i, # Name for radar plot (block number)
        r=avg_scores.values,  # Average values of the variables
        theta=avg_scores.index,  # Variable names as theta values
        mode='lines',  # Set the mode to 'lines' to remove dots on points
        fill='toself',  # Fill area inside the radar plot
        fillcolor=next(fill_color_cycle),  # Fill color is set to next color in cycle
        line=dict(color=next(line_color_cycle))  # Set the line color
    ))
    
    # Descriptive Statistics
    print(f'Block {i}\n')
    for disorder in disorders.keys():
        print(disorder)
        print(round(block_df[disorder].mean(), 3))
        print(f'({round(block_df[disorder].std(), 3)})')
        print(round(block_df[disorder].skew(), 3))
        print(round(block_df[disorder].kurtosis(), 3))
        print()
    
fig = go.Figure(data=trace_list)
    
fig.update_layout(
    title={
        'text': f'Psychopathology Domains with Binary Item-level Responses',
        'x': 0.555,  # Set x to 0.555 for alignment slightly right of center
        'y': 0.9,  # Set y to adjust the vertical position
        'font': {'size': 16}
    },
    polar=dict(
        radialaxis=dict(
            visible=True,  # Show the radial axis
            range=[0, 0.6],  # Set the range for the radial axis
            dtick=0.25
        )
    ),
    legend={
        'title': {'text': 'Block'},  # Set the title for the legend
        'title_font': {'size': 14},  # Set the font size for the legend title
        'x': 0.865,  # Adjust the horizontal position of the legend (0-1)
    }
)
fig.show()

image = pio.to_image(fig, format='png', width=8, height=8, scale=1)
with open(filepath + '.png', 'wb') as f:
    f.write(image)

# Violin plot for the youth age group (SIP Vars)
filepath = '../results/youth_SIP_scores_violin_plot'

fig, ax = plt.subplots(figsize=(6,3))

sns.set_palette(sns.color_palette('Set2'))

sns.violinplot(x='Block', y='SIP', data=youth_phenotype_df, scale='count', linewidth=2)

plt.ylim(0, 6) # Set y-axis range to 0-6

# Set title and axis labels
ax.set(xlabel='Block', ylabel='Mean Item-Level Response Value\n(Ranging from 0 to 6)')
ax.set_title('Psychosis Risk (SIP) Domain')

plt.tick_params(axis='x', which='both', bottom=False, top=False) # Remove x-axis ticks

sns.despine()

fig.savefig(filepath, bbox_inches='tight', dpi=300)



print('Early adult age group:\n')

# Radar plots and descriptive statistics for early adult age group
filepath = '../results/early_adult_symptom_scores_radar_plot'
trace_list = [] # Maintain a list of traces
for i in range(1, Q_early_adult+1):
    block_df = early_adult_phenotype_df[early_adult_phenotype_df['Block'] == i]
    avg_scores = block_df[binary_domains].mean(axis=0)
    for domain in binary_domains:
        avg_scores[domain] = avg_scores[domain] / len(disorders[domain])
    
    trace_list.append(go.Scatterpolar(
        name = i, # Name for radar plot (block number)
        r=avg_scores.values,  # Average values of the variables
        theta=avg_scores.index,  # Variable names as theta values
        mode='lines',  # Set the mode to 'lines' to remove dots on points
        fill='toself',  # Fill area inside the radar plot
        fillcolor=next(fill_color_cycle),  # Fill color is set to next color in cycle
        line=dict(color=next(line_color_cycle))  # Set the line color
    ))
    
    # Descriptive Statistics
    print(f'Block {i}\n')
    for disorder in disorders.keys():
        print(disorder)
        print(round(block_df[disorder].mean(), 3))
        print(f'({round(block_df[disorder].std(), 3)})')
        print(round(block_df[disorder].skew(), 3))
        print(round(block_df[disorder].kurtosis(), 3))
        print()
    
fig = go.Figure(data=trace_list)
    
fig.update_layout(
    title={
        'text': f'Psychopathology Domains with Binary Item-level Responses',
        'x': 0.555,  # Set x to 0.555 for alignment slightly right of center
        'y': 0.9,  # Set y to adjust the vertical position
        'font': {'size': 16}
    },
    polar=dict(
        radialaxis=dict(
            visible=True,  # Show the radial axis
            range=[0, 0.6],  # Set the range for the radial axis
            dtick=0.25
        )
    ),
    legend={
        'title': {'text': 'Block'},  # Set the title for the legend
        'title_font': {'size': 14},  # Set the font size for the legend title
        'x': 0.865,  # Adjust the horizontal position of the legend (0-1)
    }
)
fig.show()

image = pio.to_image(fig, format='png', width=8, height=8, scale=1)
with open(filepath + '.png', 'wb') as f:
    f.write(image)

# Violin plot for the early adult age group (SIP Vars)
filepath = '../results/early_adult_SIP_scores_violin_plot'

fig, ax = plt.subplots(figsize=(6,3))

sns.set_palette(sns.color_palette('Set2'))

sns.violinplot(x='Block', y='SIP', data=early_adult_phenotype_df, scale='count', linewidth=2)

plt.ylim(0, 6) # Set y-axis range to 0-6

# Set title and axis labels
ax.set(xlabel='Block', ylabel='Mean Item-Level Response Value\n(Ranging from 0 to 6)')
ax.set_title('Psychosis Risk (SIP) Domain')

plt.tick_params(axis='x', which='both', bottom=False, top=False) # Remove x-axis ticks

sns.despine()

fig.savefig(filepath, bbox_inches='tight', dpi=300)

In [ ]:
# Repeated Measures ANOVAs comparing the different domains

print('Youth age group:\n')
anova = AnovaRM(youth_data, depvar='value', subject='Block', within=['Domains'], aggregate_func='mean').fit()
print(anova.summary())

pairwise_results = pg.pairwise_ttests(dv='value', 
                                      within='Domains', 
                                      subject='Block', 
                                      data=youth_data, 
                                      padjust='fdr_bh')
print(pairwise_results)
print()

print('Early adult age group:\n')
anova = AnovaRM(early_adult_data, depvar='value', subject='Block', within=['Domains'], aggregate_func='mean').fit()
print(anova.summary())

pairwise_results = pg.pairwise_ttests(dv='value', 
                                      within='Domains', 
                                      subject='Block', 
                                      data=early_adult_data, 
                                      padjust='fdr_bh')

print(pairwise_results)

In [ ]:
# Two-tailed t-tests to compare the psychopathology symptom scores between blocks for the youth age group

file = open('../results/youth_ttest_output.txt', 'w')

for domain in list(disorders.keys()):
    file.write(f'{domain} Psychopathology Domain\n\n')
    block_disorder_df = youth_phenotype_df[['Block', domain]]
    
    sample1 = np.array(block_disorder_df[block_disorder_df['Block']==1][domain].tolist())
    sample2 = np.array(block_disorder_df[block_disorder_df['Block']==2][domain].tolist())

    # Perform the t-test
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)
    file.write(f't={t_statistic}\n')
    file.write(f'p={p_value}\n')

    # Calculate the degrees of freedom
    degrees_freedom = len(sample1) + len(sample2) - 2

    # Determine the critical value for a given alpha level (e.g., 0.05)
    alpha = 0.05
    t_critical = stats.t.ppf(1 - alpha / 2, degrees_freedom)

    # Determine if the null hypothesis is rejected or not
    if abs(t_statistic) > t_critical or p_value < alpha:
        file.write('Reject null hypothesis\n')
    else:
        file.write('Fail to reject null hypothesis\n')
    
    file.write('\n\n\n')
    
file.close()

In [ ]:
# Two-tailed t-tests to compare the psychopathology symptom scores between blocks for the early adult age group

file = open('../results/early_adult_ttest_output.txt', 'w')

for domain in list(disorders.keys()):
    file.write(f'{domain} Psychopathology Domain\n\n')
    block_disorder_df = early_adult_phenotype_df[['Block', domain]]
    
    # Assuming your data is stored in arrays or lists, you can convert them to numpy arrays
    sample1 = np.array(block_disorder_df[block_disorder_df['Block']==1][domain].tolist())
    sample2 = np.array(block_disorder_df[block_disorder_df['Block']==2][domain].tolist())

    # Perform the t-test
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)
    file.write(f't={t_statistic}\n')
    file.write(f'p={p_value}\n')

    # Calculate the degrees of freedom
    degrees_freedom = len(sample1) + len(sample2) - 2

    # Determine the critical value for a given alpha level (e.g., 0.05)
    alpha = 0.05
    t_critical = stats.t.ppf(1 - alpha / 2, degrees_freedom)

    # Determine if the null hypothesis is rejected or not
    if abs(t_statistic) > t_critical or p_value < alpha:
        file.write('Reject null hypothesis\n')
    else:
        file.write('Fail to reject null hypothesis\n')
    
    file.write('\n\n\n')
    
file.close()

In [ ]:
# Adjusted two-tailed t-tests to compare the psychopathology symptom scores between blocks for the youth age group

file = open('../results/youth_adj_ttest_output.txt', 'w')

t_stats = []
p_vals = []

domains = list(disorders.keys())

for domain in domains:
    block_disorder_df = youth_phenotype_df[['Block', domain]]
    
    # Assuming your data is stored in arrays or lists, you can convert them to numpy arrays
    sample1 = np.array(block_disorder_df[block_disorder_df['Block']==1][domain].tolist())
    sample2 = np.array(block_disorder_df[block_disorder_df['Block']==2][domain].tolist())

    # Perform the t-test
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)
    t_stats.append(t_statistic)
    p_vals.append(p_value)

# Correct for multiple comparisons using the Benjamini-Hochberg method
adj_p_vals = multipletests(p_vals, method='fdr_bh')[1]
    
for i,domain in enumerate(domains):
    file.write(f'{domain} Psychopathology Domain\n\n')
    
    t_statistic = t_stats[i]
    p_value = p_vals[i]
    adj_p_value = adj_p_vals[i]
    
    file.write(f't={t_statistic}\n')
    file.write(f'p={p_value}\n')
    file.write(f'BH-adj p={adj_p_value}\n')

    # Set the critical value
    alpha = 0.05

    # Determine if the null hypothesis is rejected or not
    if adj_p_value < alpha:
        file.write('Reject null hypothesis\n')
    else:
        file.write('Fail to reject null hypothesis\n')
    
    file.write('\n\n\n')
    
file.close()

In [ ]:
# Adjusted two-tailed t-tests to compare the psychopathology symptom scores between blocks for the early adult age group

file = open('../results/early_adult_adj_ttest_output.txt', 'w')

t_stats = []
p_vals = []

domains = list(disorders.keys())

for domain in domains:
    block_disorder_df = early_adult_phenotype_df[['Block', domain]]
    
    # Assuming your data is stored in arrays or lists, you can convert them to numpy arrays
    sample1 = np.array(block_disorder_df[block_disorder_df['Block']==1][domain].tolist())
    sample2 = np.array(block_disorder_df[block_disorder_df['Block']==2][domain].tolist())

    # Perform the t-test
    t_statistic, p_value = stats.ttest_ind(sample1, sample2)
    t_stats.append(t_statistic)
    p_vals.append(p_value)

# Correct for multiple comparisons using the Benjamini-Hochberg method
adj_p_vals = multipletests(p_vals, method='fdr_bh')[1]
    
for i,domain in enumerate(domains):
    file.write(f'{domain} Psychopathology Domain\n\n')
    
    t_statistic = t_stats[i]
    p_value = p_vals[i]
    adj_p_value = adj_p_vals[i]
    
    file.write(f't={t_statistic}\n')
    file.write(f'p={p_value}\n')
    file.write(f'BH-adj p={adj_p_value}\n')

    # Set the critical value
    alpha = 0.05

    # Determine if the null hypothesis is rejected or not
    if adj_p_value < alpha:
        file.write('Reject null hypothesis\n')
    else:
        file.write('Fail to reject null hypothesis\n')
    
    file.write('\n\n\n')
    
file.close()

In [ ]:
# Multivariate ANOVA
manova = MANOVA.from_formula('ADD + AGR + CDD + DEP + GAD + MAN + OCD + ODD + PAN + PHB + PSY + SIP + PTD + SEP + SOC + SUI ~ age_group + Block + age_group:Block', data=data)
print(manova.mv_test())

# Visualization of the optimal model selection process for the multiplex SBM fit to a neuroimaging layer constructed using pairwise Pearson dissimilarities

In [ ]:
# Visualization for model selection based on ICL criterion
youth_stored_models = pd.read_csv('../results/youth_multiplex_Pearson_stored_models.csv')
youth_stored_models = youth_stored_models[['nbBlocks', 'ICL']].sort_values('nbBlocks')

early_adult_stored_models = pd.read_csv('../results/early_adult_multiplex_Pearson_stored_models.csv')
early_adult_stored_models = early_adult_stored_models[['nbBlocks', 'ICL']].sort_values('nbBlocks')

max_youth = np.argmax(youth_stored_models['ICL'].to_list())
max_early_adult = np.argmax(early_adult_stored_models['ICL'].to_list())

fig, ax = plt.subplots(figsize=(6,3))

ax.plot(youth_stored_models['nbBlocks'].to_list(), youth_stored_models['ICL'].to_list(), '-o', color='tab:blue', label='Youth')
ax.plot(early_adult_stored_models['nbBlocks'].to_list(), early_adult_stored_models['ICL'].to_list(), '-o', color='tab:orange', label='Early Adults')

ax.plot(len(youth_stored_models['nbBlocks']) + 1, youth_stored_models['ICL'].to_list()[-1], 'x', color='red', label='Failed to Converge')
ax.plot(len(early_adult_stored_models['nbBlocks']) + 1, early_adult_stored_models['ICL'].to_list()[-1], 'x', color='red')

ax.scatter(max_youth+1, youth_stored_models['ICL'].iloc[max_youth], s=200, marker='*', edgecolors='tab:blue', facecolors='tab:blue',
           label='$Opt_{Youth}$ = '+str(round(youth_stored_models['ICL'].iloc[max_youth], 2)))
ax.scatter(max_early_adult+1, early_adult_stored_models['ICL'].iloc[max_early_adult], s=200, marker='*', edgecolors='tab:orange', facecolors='tab:orange',
           label='$Opt_{Early Adult}$ = '+str(round(early_adult_stored_models['ICL'].iloc[max_early_adult], 2)))

ax.set(xlabel='Number of Blocks (Q)', ylabel='Integrated Completed Likelihood (ICL)')
ax.set_title('Selection of Optimal Q', y=1.075)

x_ticks = [1, 2, 3, 4, 5]
plt.xticks(x_ticks)

plt.legend(loc='upper right', bbox_to_anchor=(1.45, 1))

plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
formatter = ticker.ScalarFormatter(useMathText=True)
formatter.set_scientific(True)
formatter.set_powerlimits((0, 0))
plt.gca().yaxis.set_major_formatter(formatter)

sns.despine()

filepath = '../results/multiplex_SBM_Pearson_ICL'

fig.savefig(filepath, bbox_inches='tight', dpi=300)